# Building a Spam Filter with Naive Bayes

<img src="https://media.istockphoto.com/id/1323226102/photo/cyber-security-phishing-e-mail-network-security-computer-hacker-cloud-computing-ransomware.jpg?s=612x612&w=0&k=20&c=owkSqKKwVO94cN8WFmt4LA72cBb1kLVa8wguopJ8OhM=" width="800" height="100">


In this project, we'll be creating a spam filter for SMS messages using the multinomial Naive Bayes algorithm. Our objective is to design a program that accurately classifies new messages with over 80% precision, ensuring that more than 80% of the new messages are correctly categorized as spam or ham (non-spam).

To train the algorithm, we'll be utilizing a dataset of 5,572 SMS messages that have already been manually classified by humans. This dataset was expertly curated by Tiago A. Almeida and José María Gómez Hidalgo and is available for download from [The UCI Machine Learning Repository](https://archive-beta.ics.uci.edu/datasets?search=SMS%20Spam%20Collection). For more information on the data collection process and access to papers authored by Tiago A. Almeida and José María Gómez Hidalgo, please visit [this page](https://www.researchgate.net/publication/258514273_Towards_SMS_Spam_Filtering_Results_under_a_New_Dataset).

## Exploring the Dataset

To begin, we can read the `SMSSpamCollection` dataset using the `read_csv()` function from the `Pandas` package.

It's important to note that the data points are separated by tabs, so we'll need to use the `sep='\t'` parameter when calling the `read_csv()` function. Additionally, as the dataset lacks a header row, we need to use the `header=None` parameter to avoid the first row being incorrectly interpreted as the header row.

Finally, to name the columns `Label` and `SMS`, we can use the `names=['Label', 'SMS']` parameter in our `read_csv()` function call.

In [1]:
import pandas as pd

# Turn off the SettingWithCopyWarning
pd.options.mode.chained_assignment = None

# Read dataset
sms_collection = pd.read_csv('SMSSpamCollection', sep='\t', names=['Label', 'SMS'])

# View first few rows
sms_collection.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [2]:
# Find number of rows and columns
print(f'Number of rows: {sms_collection.shape[0]}')
print(f'Number of columns: {sms_collection.shape[1]}')

Number of rows: 5572
Number of columns: 2


In [3]:
# Find percentage of messages label (spam and ham)
round(sms_collection['Label'].value_counts(normalize=True)*100)

ham     87.0
spam    13.0
Name: Label, dtype: float64

Looking at the data, we can observe that approximately 87% of the messages are categorized as ham, with the remaining 13% being classified as spam. This sample appears to be a fair representation since, in reality, the majority of messages that individuals receive are typically ham.

## Training and Test Set

Before building our spam filter, it's important to think about how we'll test its effectiveness. This means designing a test before creating the software to avoid bias.

To test our spam filter, we'll split our dataset into two categories: a training set and a test set.

- The **training set** will have 80% of the dataset, which amounts to 4,458 messages.
- The **test set** will have 20% of the dataset, which amounts to 1,114 messages.

All 1,114 messages in our test set are already classified by a human. When the spam filter is ready, we'll treat these messages as new and have the filter classify them.

Our goal is to create a spam filter that can classify new messages with an accuracy greater than 80%. We expect that more than 80% of the new messages will be classified correctly as spam or ham.

To ensure that spam and ham messages are spread properly throughout the dataset, we'll randomize the entire dataset before creating the training and test sets.

In [4]:
# Randomize the dataset and reset index
data_randomized = sms_collection.sample(frac=1, random_state=1).reset_index(drop=True)

# Calculate the training size
training_size = int(len(data_randomized) * 0.8)

# Split the dataset into training (80%) and test set (20%)
training_set = data_randomized.iloc[:training_size]
test_set = data_randomized.iloc[training_size:]

# Verify the sizes of the training and test sets
print(f'Number of messages in the training set: {len(training_set)}')
print(f'Number of messages in the test set: {len(test_set)}')

Number of messages in the training set: 4457
Number of messages in the test set: 1115


Next, we'll calculate the percentages of spam and ham messages in both the training and test sets. We'll compare these percentages to the ones in the full dataset to see if they are similar.

In [5]:
round(training_set['Label'].value_counts(normalize=True)*100)

ham     87.0
spam    13.0
Name: Label, dtype: float64

In [6]:
round(test_set['Label'].value_counts(normalize=True)*100)

ham     87.0
spam    13.0
Name: Label, dtype: float64

The percentages in both the training and test set are similar to those in the full dataset, where the majority of messages are ham, accounting for about 87%, and the remaining 13% are spam. The obtained results confirm our expectations. With this done, we can proceed to cleaning the dataset.

## Data Cleaning

In the previous step, we divided our data into two sets: a training set and a test set. The next step is to use the training set to teach our algorithm how to classify new messages.

When a new message arrives, the Naive Bayes algorithm uses the following two equations to classify it (note that "Ham" refers to "non-spam" in the second equation):

$$P(Spam|w_1,w_2,...,w_n) \propto P(Spam) \cdot \prod\limits_{i=1}^{n} P(w_i|Spam)$$
$$P(Ham|w_1,w_2,...,w_n) \propto P(Ham) \cdot \prod\limits_{i=1}^n P(w_i|Ham)$$

To calculate the values for $P(wi|Spam)$ and $P(wi|Ham)$, we use these equations:

$$P(w_i|Spam) = \frac{N_{w_i|Spam}+\alpha}{N_{Spam}+\alpha \cdot N_{Vocabulary}}$$
$$P(w_i|Ham) = \frac{N_{w_i|Ham}+\alpha}{N_{Ham}+\alpha \cdot N_{Vocabulary}}$$

Here's what the terms in the equations mean:

$N_{w_i|Spam}$ = the number of times the word $w_i$ appears in spam messages

$N_{w_i|Ham}$ = the number of times the word $w_i$ appears in non-spam messages

$N_{Spam}$ = total number of words in spam messages

$N_{Ham}$ = total number of words in non-spam messages

$N_{Vocabulary}$ = total number of words in the vocabulary (vocabulary is the set of all unique words)

$\alpha$ = 1 ($\alpha$ is a smoothing parameter)

To calculate all these probabilities, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need. Right now, our training and test sets have this format (the example messages are not real and are used for clarity):

| Label | SMS |
|-------|---------|
| spam  | SECRET PRIZE! CLAIM SECRET PRIZE NOW!! |
| ham   | Coming to my secret party? |
| spam  | Winner! Claim secret prize now! |

To make the calculations easier, we want bring the data to this format (the table below is a transformation of the table we see above):

| Label | secret | prize | claim | now | coming | to  | my  | party | winner |
|-------|--------|-------|-------|-----|--------|-----|-----|-------|--------|
| spam  | 2    | 2    | 1    | 1   | 0      | 0   | 0   | 0     | 0     |
| ham  | 1     | 0   | 0    | 0   | 1      | 1   | 1   | 1     | 0    |
| spam   | 1     | 1    | 1   | 1   | 0      | 0   | 0   | 0     | 1     |

About the transformation above, notice that:

- The `SMS` column doesn't exist anymore.
- Instead, the `SMS` column is replaced by a series of new columns, where each column represents a unique word from the vocabulary.
- Each row describes a single message. For instance, the first row corresponds to the message "SECRET PRIZE! CLAIM SECRET PRIZE NOW!!", and it has the values `spam, 2, 2, 1, 1, 0, 0, 0, 0, 0`. These values tell us that:

    - The message is spam.
    - The word "secret" occurs two times inside the message.
    - The word "prize" occurs two times inside the message.
    - The word "claim" occurs one time inside the message.
    - The word "now" occurs one time inside the message.
    - The words "coming", "to", "my", "party", and "winner" occur zero times inside the message.

- All words in the vocabulary are in lower case, so "SECRET" and "secret" come to be considered to be the same word.
- Punctuation is not taken into account anymore (for instance, we can't look at the table and conclude that the first message initially had three exclamation marks).

### Letter Case and Punctuation

Let's begin the data cleaning process on the training set by removing the punctuation marks and converting all the words to lower case.

In [7]:
# After cleaning
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ', regex=True).str.lower()
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


### Creating the Vocabulary

Our goal is to transform the training set into a table where each column represents a unique word in our **vocabulary**, and each row represents a message, with the values in each cell indicating the frequency of that word in that message. To do that, we first need to create a list of all the unique words in our training set, excluding the `Label` column.

In [8]:
# Split each message in the `SMS` column at the space character to transform it into a list
training_set['SMS'] = training_set['SMS'].str.split()

# List to store strings from 'SMS'
vocabulary = []

# Loop through the 'SMS' column and add each string to the list 'vocabulary'
for str_list in training_set['SMS']:
    for string in str_list:
        vocabulary.append(string)
        
print(f'List size before removing duplicates: {len(vocabulary)}')

List size before removing duplicates: 72423


We'll remove duplicates from the `vocabulary` list by transforming it into a set using the `set()` function. Then, we'll convert the resulting set back into a list using the `list()` function.

In [9]:
vocabulary = list(set(vocabulary))
print(f'List size after removing duplicates: {len(vocabulary)}')

List size after removing duplicates: 7782


### The Final Training Set

After creating the vocabulary for our training set messages, we will use it to transform the data to a format that helps us to count the frequency of each unique word in the messages.

***From***: 

| Label | SMS |
|-------|---------|
| spam  | SECRET PRIZE! CLAIM SECRET PRIZE NOW!! |
| ham   | Coming to my secret party? |
| spam  | Winner! Claim secret prize now! |

***To***:

| Label | secret | prize | claim | now | coming | to  | my  | party | winner |
|-------|--------|-------|-------|-----|--------|-----|-----|-------|--------|
| spam  | 2    | 2    | 1    | 1   | 0      | 0   | 0   | 0     | 0     |
| ham  | 1     | 0   | 0    | 0   | 1      | 1   | 1   | 1     | 0    |
| spam   | 1     | 1    | 1   | 1   | 0      | 0   | 0   | 0     | 1     |

Our end goal is to transform our training set into a new DataFrame that shows the frequency of each unique word in each message. However, before we do that, we need to build a dictionary that will hold this information. We'll then use this dictionary to create the DataFrame we need.

For instance, to create the table we see above, we could use this dictionary and then convert it to a DataFrame:

```python
word_counts_per_sms = {'secret': [2,1,1],
                       'prize': [2,0,1],
                       'claim': [1,0,1],
                       'now': [1,0,1],
                       'coming': [0,1,0],
                       'to': [0,1,0],
                       'my': [0,1,0],
                       'party': [0,1,0],
                       'winner': [0,0,1]
                      }


word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()
```

|   | secret | prize | claim | now | coming | to | my | party | winner |
|---|--------|-------|-------|-----|--------|----|----|-------|--------|
| 0 | 2      | 2     | 1     | 1   | 0      | 0  | 0  | 0     | 0      |
| 1 | 1      | 0     | 0     | 0   | 1      | 1  | 1  | 1     | 0      |
| 2 | 1      | 1     | 1     | 1   | 0      | 0  | 0  | 0     | 1      |


Note that the `Label` column is missing in the above table but we'll get to that later.

To create the dictionary we need for our training set, we'll take the following steps:

- We start by initializing a dictionary named `word_counts_per_sms`, where each key is a unique word (a string) from the **vocabulary**, and each value is a list of the length of training set, where each element in the list is a `0`.
    - The code `[0] * 5` outputs `[0, 0, 0, 0, 0]`. So the code `[0] * len(training_set['SMS'])` outputs a list of the length of `training_set['SMS']`, where each element in the list will be a `0`.
- We loop over `training_set['SMS']` using at the same time the [`enumerate()` function](https://docs.python.org/3/library/functions.html#enumerate) to get both the index and the SMS message (`index` and `sms`).
    - Using a nested loop, we loop over `sms` (where `sms` is a list of strings, where each string represents a word in a message).
        - We incremenent `word_counts_per_sms[word][index]` by `1`.

In [10]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1 # increment index number

Let's transform `word_counts_per_sms` into a DataFrame using `pd.DataFrame()`.

In [11]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,wallet,invitation,cabin,enough,everyday,transfred,tmorrow,slide,studio,fondly,...,enufcredeit,give,steak,09064011000,snogs,gave,lift,eightish,celebrations,beg
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Concatenate the DataFrame containing the word counts with the DataFrame of the training set to get the `Label` and `SMS` columns as well using the [`pd.concat()` function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html).

In [12]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,wallet,invitation,cabin,enough,everyday,transfred,tmorrow,slide,...,enufcredeit,give,steak,09064011000,snogs,gave,lift,eightish,celebrations,beg
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating Constants First

To classify new messages as spam or ham, we need to calculate the probability values of two equations below:

$$P(Spam|w_1,w_2,...,w_n) \propto P(Spam) \cdot \prod\limits_{i=1}^{n} P(w_i|Spam)$$
$$P(Ham|w_1,w_2,...,w_n) \propto P(Ham) \cdot \prod\limits_{i=1}^n P(w_i|Ham)$$

We'll use the following equations to calculate $P(w_i|Spam)$ and $P(w_i|Ham)$:

$$P(w_i|Spam) = \frac{N_{w_i|Spam}+\alpha}{N_{Spam}+\alpha \cdot N_{Vocabulary}}$$
$$P(w_i|Ham) = \frac{N_{w_i|Ham}+\alpha}{N_{Ham}+\alpha \cdot N_{Vocabulary}}$$

Some of the terms in the four equations above will have the same value for every new message. As a start, let's first calculate:

- $P(Spam)$ and $P(Ham)$
- $N_{Spam}$, $N_{Ham}$, $N_{Vocabulary}$

Where:

- $N_{Spam}$ is equal to the number of words in all the spam messages.
- $N_{Ham}$ is equal to the number of words in all the non-spam messages.
- $N_{Vocabulary}$ is equal to the total number of unique words in the training set.

We'll also use Laplace smoothing and set $\alpha$ = 1

In [13]:
# Filter rows with spam and ham messages
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']

# Calculate probability of spam and ham in the training set
p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)

# Calculate number of words in spam, ham, and vocabulary
n_spam = spam_messages['SMS'].apply(len).sum()
n_ham = ham_messages['SMS'].apply(len).sum()

# Calculate total number of unique words in the training set
n_vocabulary = len(vocabulary)

# Set Laplace smoothing
alpha = 1

## Calculating Parameters

Once we have the constant terms calculated above, we can calculate the parameters $P(w_i|Spam)$ and $P(w_i|Ham)$ using the following formulas. Each parameter is a conditional probability value associated with each word in the vocabulary.

$$P(w_i|Spam) = \frac{N_{w_i|Spam}+\alpha}{N_{Spam}+\alpha \cdot N_{Vocabulary}}$$

$$P(w_i|Ham) = \frac{N_{w_i|Ham}+\alpha}{N_{Ham}+\alpha \cdot N_{Vocabulary}}$$

In [14]:
# Initiate parameters
parameters_spam = {unique_word: 0 for unique_word in vocabulary}
parameters_ham = {unique_word: 0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

## Classifying A New Message

Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:

- Takes in as input a new message $(w_1, w_2, ..., w_n)$
- Calculates $P(Spam|w_1, w_2, ..., w_n)$ and $P(Ham|w_1, w_2, ..., w_n)$
- Compares the values of $P(Spam|w_1, w_2, ..., w_n)$ and $P(Ham|w_1, w_2, ..., w_n)$, and:
    - If $P(Ham|w_1, w_2, ..., w_n) > P(Spam|w_1, w_2, ..., w_n)$, then the message is classified as ham.
    - If $P(Ham|w_1, w_2, ..., w_n) < P(Spam|w_1, w_2, ..., w_n)$, then the message is classified as spam.
    - If $P(Ham|w_1, w_2, ..., w_n) = P(Spam|w_1, w_2, ..., w_n)$, then the algorithm may request human help.

In [15]:
import re

def classify(message):
    """
    Classify a given message as "Spam" or "Ham" using a Naive Bayes algorithm.

    Parameters:
    -----------
    message : str
        The message to be classified.

    Returns:
    --------
    None
        This function only prints the label assigned to the message.

    Example:
    --------
    >>> classify("Urgent! Call now to claim your prize!")
    P(Spam|message): 0.999
    P(Ham|message): 0.001
    Label: Spam
    """

    
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
        if word not in parameters_spam and word not in parameters_ham:
            pass
    
    print(f'P(Spam|message): {p_spam_given_message}')
    print(f'P(Ham|message): {p_ham_given_message}')
    
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

We can test the `classify()` function by passing two new messages as arguments:

In [16]:
test_msgs = ['WINNER!! This is the secret code to unlock the money: C3421.', 'Sounds good, Tom, then see u there']

for msg in test_msgs:
    classify(msg)
    print()

P(Spam|message): 1.3489598779101096e-25
P(Ham|message): 1.9380782419077522e-27
Label: Spam

P(Spam|message): 2.4385273359614485e-25
P(Ham|message): 3.6893872875947e-21
Label: Ham



## Measuring the Spam Filter's Accuracy

In the previous section, we developed a spam filter and used it to classify two new messages. In the following step, we will evaluate the performance of our spam filter on a test set containing 1,115 messages. We will compare the output classification labels generated by the algorithm to the actual labels provided by humans. It is worth noting that during training, our algorithm was not exposed to these 1,115 messages, making every message in the test set novel from the algorithm's perspective.

To begin with, we need to modify the `classify()` function that we created earlier to return the labels instead of printing them. As shown below, the `return` statements replace the `print()` functions.

In [17]:
def classify_test_set(message):
    
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
        if word not in parameters_spam and word not in parameters_ham:
            pass
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

Now that we have a function that returns labels instead of printing them, we can use it to create a new column in our test set.

In [18]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
4457,ham,Wherre's my boytoy ? :-(,ham
4458,ham,Later i guess. I needa do mcat study too.,ham
4459,ham,But i haf enuff space got like 4 mb...,ham
4460,spam,Had your mobile 10 mths? Update to latest Oran...,spam
4461,ham,All sounds good. Fingers . Makes it difficult ...,ham


Now we can compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use **accuracy** as a metric:

$$\text{Accuracy}=\frac{\text{number of correctly classified messages}}{\text{total number of classified messages}}$$

In [19]:
correct = 0
total = test_set.shape[0]

for index, row in test_set.iterrows():
    if row['Label'] == row['predicted']:
        correct += 1
        
print(f'Correct: {correct}')
print(f'Incorrect: {total - correct}')
print(f'Accuracy: {round(correct/total*100, 2)}')

Correct: 1101
Incorrect: 14
Accuracy: 98.74


Our spam filter achieved an accuracy of 98.74%, which is quite impressive. Out of the 1,115 messages in the test set, the filter correctly classified 1,101 of them.

## Conclusion

In conclusion, this project successfully implemented a spam filter using the multinomial Naive Bayes algorithm and a dataset of 5,572 labeled SMS messages. The filter is capable of classifying a new message as either spam or ham with high accuracy, achieving a rate of 98.74%, which is significantly better than our initial target.